# Notch Filter Algorithm

In [ ]:
import cv2
# to install cv2 use: pip install opencv-python
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
# shape: (W,H) are the width and height of the filter

# d0 is the radius of the filter expressed in pixels

# u_k and v_k are the center coordinates in the Fourier image plane for the notch filter
# u_k=0, v_k=0 correspond to the center of the shifted spectrum

def notch_reject_filter_vec(shape: tuple[int, int], d0: int, u_k: int, v_k: int):
    (W, H) = shape

    H_0_u = np.repeat(np.arange(W), H).reshape((W, H))
    H_0_v = np.repeat(np.arange(H), W).reshape((H, W)).transpose()

    D_uv = np.sqrt((H_0_u - W / 2 + u_k) ** 2 + (H_0_v - H / 2 + v_k) ** 2)
    D_muv = np.sqrt((H_0_u - W / 2 - u_k) ** 2 + (H_0_v - H / 2 - v_k) ** 2)

    selector_1 = D_uv <= d0
    selector_2 = D_muv <= d0

    selector = np.logical_or(selector_1, selector_2)

    H = np.ones((W, H, 2), np.uint8)
    H[selector] = 0

    return H

In [ ]:
def notchFilter(img):
    img_float32 = np.float32(img)

    # compute the DFT
    # Note that the computed DFT has 2 channels :
    # dft[:,:,0] - real values
    # dft[:,:,1] - imaginary values
    dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
    # recenter the DFT , such that the center is (0,0)
    dft_shift = np.fft.fftshift(dft)

    print('DFT Shape =', dft_shift.shape)

    # compute the frequency spectrum
    # module of the complex value
    magnitude_spectrum = cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1])
    # compute the logarith of magnitude such that smaller values are visible
    magnitude_spectrum = 20*np.log(magnitude_spectrum)

    # several notch filters can be composited by multiplication

    # fill in the following lists to compose multiple notch filters

    d0_list = [8,8,5,5]                 # raio do nodo
    u_k_list = [20,20,-2,14]            # coordenada u do nodo (y)
    v_k_list = [25,-25,-20,-4]          # coordenada v do nodo (x)

    H = notch_reject_filter_vec(img.shape, d0_list[0], u_k_list[0], v_k_list[0])

    for d0, u_k, v_k in zip (d0_list[1:], u_k_list[1:], v_k_list[1:]):
        Haux = notch_reject_filter_vec(img.shape, d0, u_k, v_k)
        H *= Haux

    # apply mask and inverse DFT<
    dft_masked = dft_shift * H
    # do not forget to undo the centering of the transform
    dft_masked_ishift = np.fft.ifftshift(dft_masked)
    img_back = cv2.idft(dft_masked_ishift)

    # img_back should only have real values
    # but let's make sure due to rounding and representation errors
    img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])

    # compute the DFT of the processed image (img_back)
    # Note that the computed DFT has 2 channels :
    # dft[:,:,0] - real values
    # dft[:,:,1] - imaginary values
    dft_proc = cv2.dft(img_back, flags = cv2.DFT_COMPLEX_OUTPUT)
    # recenter the DFT , such that the center is (0,0)
    dft_proc_shift = np.fft.fftshift(dft_proc)
    # compute the frequency spectrum
    # module of the complex value
    filtered_spectrum = cv2.magnitude(dft_proc_shift[:,:,0],dft_proc_shift[:,:,1])
    # compute the logarith of magnitude such that smaller values are visible
    filtered_spectrum = 20*np.log(filtered_spectrum)

    # compute the frequency spectrum of the MASK, 
    # just for display purposes
    # module of the complex value
    mask_spectrum = np.sqrt(np.power(H[:,:,0],2)+np.power(H[:,:,1],2))
    # compute the logarith of magnitude such that smaller values are visible
    mask_spectrum = 20*np.log(mask_spectrum+0.0001)


    # display the image and the spectrum
    fig = plt.figure(figsize=(15,23))
    plt.subplot(321),plt.imshow(img, cmap = 'gray')
    plt.title('Input Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(322),plt.imshow(magnitude_spectrum, cmap = 'gray')
    plt.title('Input Spectrum'), plt.xticks([]), plt.yticks([])
    plt.subplot(323),plt.imshow(mask_spectrum, cmap = 'gray')
    plt.title('Filter Spectrum'), plt.xticks([]), plt.yticks([])
    plt.subplot(325),plt.imshow(img_back, cmap = 'gray')
    plt.title('Filtered Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(326),plt.imshow(filtered_spectrum, cmap = 'gray')
    plt.title('Filtered Spectrum'), plt.xticks([]), plt.yticks([])
    plt.show()

    return img_back         